In [ ]:
import csv
import os
import time
import pyodbc    # pip install pyodbc

# 1. Connect to Azure SQL Database

In [ ]:
# Define database connection details
server   = 'AZURE_SQL_DATABASE_SERVER_NAME'
database = 'AZURE_SQL_DATABASE_DATABASE_NAME'
username = 'AZURE_SQL_DATABASE_USERNAME'
password = 'AZURE_SQL_DATABASE_PASSWORD'
driver   = '{ODBC Driver 17 for SQL Server}'

# Connect to Azure SQL Database
connection = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password};'
)

# Create a cursor
cursor = connection.cursor()

# 2. Retrieve number of affected datasets by the actual CRUD-Operation

In [ ]:
sql_query = """
SELECT COUNT(DISTINCT io.SerialNumber)
FROM InspectionOperations io
WHERE io.ArticleName = 'Mountainbike' AND io.MachineName = 'InspectionMachine1';
"""

# Execute the SQL Query and store the number of affected datasets
cursor.execute(sql_query)
number_of_processed_datasets = cursor.fetchone()[0]

# 3. Run the operation

In [ ]:
# Define a list to store the operation durations for each dataset that is inserted
query_durations = []

# Number of datasets in the database (e.g., 10,000 after the first iteration)
database_record_count = 10000

In [ ]:
# Run this use case 10 times per iteration
for _ in range(10):

    sql_query = f"""
    SELECT 
        *
    FROM 
        InspectionOperations io
    JOIN 
        Inspections i ON io.InspectionOperationID = i.InspectionOperationID
    JOIN 
        InspectionResults ir ON i.InspectionID = ir.InspectionID
    JOIN 
        InspectionsToSetpoints its ON i.InspectionID = its.InspectionID
    JOIN 
        Setpoints isp ON its.SetpointID = isp.SetpointID
    JOIN 
        InspectionStep ins ON i.InspectionStepID = ins.InspectionStepID
    WHERE 
        io.ArticleName = 'Mountainbike' AND io.MachineName = 'InspectionMachine1'
    """

    # Record the current timestamp before running the operation  
    query_start_time = time.time()
    
    # Run CRUD-Operation
    cursor.execute(sql_query)
    
    # Record the current timestamp after running the operation
    query_end_time = time.time()

    # Calculate the duration time for this operation
    query_duration = query_end_time - query_start_time
    query_durations.append(query_duration)

# 4. Saving the recorded operation times in the CSV result file

In [ ]:
# Calculate the average duration of all operations in this iteration
mean_duration = sum(query_durations) / len(query_durations)

# Define the dataset to store
dataset_to_store = [[
    mean_duration,                # Average duration of operations in this iteration
    number_of_processed_datasets, # Number of processed datasets
    database_record_count         # Current number of datasets in the database
]]

# Store values in the CSV result file
filepath = os.path.join("Experiment_Results", "select_to_serialnumber.csv")
file_exists = os.path.isfile(filepath)

with open(filepath, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write header if the file does not exist
    if not file_exists:
        writer.writerow(['DurationTime', 'NumberOfProcessedDatasets', 'NumberOfDatasetsInDatabase'])
    
    # Append the dataset to the CSV file
    writer.writerows(dataset_to_store)